In [5]:
#Before this program can be run, the user must first install the Zomato sdk to the Anaconda Prompt.
#Use the command 'pip install zomato-sdk' to prepare for the program.

from zomato import Zomato
import requests
import json
import pandas as pd
import folium
import warnings
warnings.filterwarnings('ignore')

#The code "headers = {'Accept': 'application/json', 'user-key': 'bc7aaa1e5521c5a02ed53a89a891ae20'}"
#was translated from curl.trillworks.com
#An individual that wants to use this program on their own system should generate their own user-key from the Zomato API.

def citylookup(city):
    headers = {'Accept': 'application/json', 'user-key': 'bc7aaa1e5521c5a02ed53a89a891ae20'}
    city_url = 'https://developers.zomato.com/api/v2.1/cities?q=' + city
    city_response = requests.get(city_url, headers=headers).json()
    return city_response

def getcityid(city):
    city_func_response = citylookup(city)
    city_id = city_func_response['location_suggestions'][0]['id']
    city_id = str(city_id)
    return city_id

def getlocationdetails(city_id):
    headers = {'Accept': 'application/json', 'user-key': 'bc7aaa1e5521c5a02ed53a89a891ae20'}
    location_details_url = 'https://developers.zomato.com/api/v2.1/location_details?entity_id=' + city_id + '&entity_type=city'
    location_details_response = requests.get(location_details_url, headers=headers).json()
    return location_details_response

try:
    print("If the map returns the incorrect city, try typing the full state name. (ex. Albany, New York)")
    city = input("Please enter your city and state (ex. Syracuse, NY): ")
    print("\n\nClick the markers on the map to see the name of the top-rated restaurant.")
    print("The reviews for each restaurant are out of '5'.")
    city_id = getcityid(city)
    city_details = getlocationdetails(city_id)
    i = 0
    restaurant_list = []
    lat_list = []
    long_list = []
    review_list = []

    for item in city_details['best_rated_restaurant']:
        restaurant_list.append(city_details['best_rated_restaurant'][i]['restaurant']['name'])
        lat_list.append(city_details['best_rated_restaurant'][i]['restaurant']['location']['latitude'])
        long_list.append(city_details['best_rated_restaurant'][i]['restaurant']['location']['longitude'])
        review_list.append(city_details['best_rated_restaurant'][i]['restaurant']['user_rating']['aggregate_rating'])
        i = i + 1

    for i in range(len(lat_list)):
        lat_list[i] = float(lat_list[i])
    for i in range(len(long_list)):
        long_list[i] = float(long_list[i])
    
#The first 3 exceptions were taken directly from CCL 11 in Michael Fudge's class.    
    
except json.decoder.JSONDecodeError as e: 
    print("ERROR: Cannot decode the response into json")
    print("DETAILS", e)

except requests.exceptions.HTTPError as e:
    print("ERROR: Response was not ok.")
    print("DETAILS:", e)
        
except requests.exceptions.RequestException as e: 
    print("ERROR: Cannot connect to the site at this time.")
    print("DETAILS:", e)
    
except:
    print("\nSorry. The results could not be processed.")
    print("Please check that you spelled the city correctly, or type in a different city.")   
    
restaurants = pd.Series(data = restaurant_list)
latitudes = pd.Series(data = lat_list)
longitudes = pd.Series(data = long_list)
user_ratings = pd.Series(data = review_list)
city_dict = {'City': restaurants, 'Latitude': latitudes, 'Longitude': longitudes, 'Average Rating': user_ratings}
city_df = pd.DataFrame(city_dict)    

first_lat = float(city_df['Latitude'][0])
first_long = float(city_df['Longitude'][0])
map_center = (first_lat, first_long)
map = folium.Map(location = map_center, zoom_start = 12)
for row in city_df.to_records():
    pos = (row['Latitude'],row['Longitude'])
    marker = folium.Marker(location = pos, popup = folium.Popup(row['City'] + ' -- Average Rating: ' + row['Average Rating'], parse_html=True))
    map.add_children(marker)
map

If the map returns the incorrect city, try typing the full state name. (ex. Albany, New York)
Please enter your city and state (ex. Syracuse, NY): Syracuse


Click the markers on the map to see the name of the top-rated restaurant.
The reviews for each restaurant are out of '5'.
